# 1. Library Installation

In [ ]:

import os
import json
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document



# 2. Define paths

In [ ]:
PROJECT_ROOT = "Fake_Job_Posting_Detection"

RAW_DATA_PATH = os.path.join(
    PROJECT_ROOT, "data", "raw", "fake_job_postings.csv"
)

RAG_STORE_PATH = os.path.join(PROJECT_ROOT, "rag_store")
FAISS_INDEX_PATH = os.path.join(RAG_STORE_PATH, "faiss_index")
EMBEDDING_CONFIG_PATH = os.path.join(RAG_STORE_PATH, "embedding_config.json")

os.makedirs(RAG_STORE_PATH, exist_ok=True)


# 3. Load RAW dataset

In [ ]:
df_raw = pd.read_csv(RAW_DATA_PATH)

required_cols = [
    "title", "company_profile", "description", "requirements", "fraudulent"
]
assert all(col in df_raw.columns for col in required_cols)

print(f"Raw dataset loaded: {df_raw.shape}")


# 4. Filter legitimate job postings ONLY
**Design rule:**
RAG explanations must be grounded in verified real jobs

In [ ]:
real_jobs_df = (
    df_raw[df_raw["fraudulent"] == 0]
    .sample(n=300, random_state=42)
    .reset_index(drop=True)
)

print(f"Real jobs selected: {len(real_jobs_df)}")


# 5. Hugging Face embedding model

In [ ]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

embedding_function = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)

print(f"Embedding model configured: {EMBEDDING_MODEL_NAME}")



# 6. Document construction

In [ ]:
def build_job_document(row):
    return (
        f"Title: {row['title']}. "
        f"Company Profile: {row['company_profile']}. "
        f"Job Description: {row['description']}. "
        f"Requirements: {row['requirements']}."
    )


# 7. Create LangChain documents

In [ ]:
documents = [
    Document(
        page_content=build_job_document(row),
        metadata={"title": row["title"]}
    )
    for _, row in real_jobs_df.iterrows()
]

print(f"Documents created: {len(documents)}")


# 8. Build & persist FAISS index

In [ ]:
faiss_store = FAISS.from_documents(documents, embedding_function)

faiss_store.save_local(FAISS_INDEX_PATH)

print("FAISS index saved successfully")


# 9. Save embedding metadata

In [ ]:
embedding_config = {
    "embedding_model": EMBEDDING_MODEL_NAME,
    "num_documents": len(documents),
    "data_source": "raw_fake_job_postings.csv",
    "document_policy": "real_jobs_only"
}

with open(EMBEDDING_CONFIG_PATH, "w") as f:
    json.dump(embedding_config, f, indent=4)

print("Embedding configuration saved")


In [ ]:
assert os.path.exists(FAISS_INDEX_PATH)
assert os.path.exists(EMBEDDING_CONFIG_PATH)

print("RAG artifacts built successfully")